In [2]:
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=4affbc1abb1b89c7bc0cfc6d8d332039f55637a669aead6045deb777d4056b7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-w4j1vbfx/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
     |████████████████████████████████| 12.0 MB 5.6 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [3]:
import pandas as pd


In [4]:
import spacy 
nlp = spacy.load('en_core_web_sm')

In [6]:
faq = pd.read_csv('/content/faq.csv')
faq.head(10)

,question,answer,category
0,What if I need to know my correct CGPA?,CGPA is seen on the grade card. The CGPA count...,grade card
1,I have cleared my backlog but the result sheet...,GPA and CGPA are not mentioned on backlog gra...,grade card
2,"I have cleared my backlog, but the result on E...",Grade card shows first attempt GPA and CGPA o...,grade card
3,What do I do if I have not received my grade c...,Grade cards should be collected from the stude...,grade card
4,My name/Father's Name/Mother's name is incorre...,Name correction on grade cards/PDC needs to b...,Corrections on grade card /PDC
5,How can I get a new copy of corrected grade ca...,Once the required corrections are done in ERP...,Corrections on grade card /PDC
6,From where should I collect the corrected grad...,You have to collect the corrected grade cards/...,Corrections on grade card /PDC
7,In how many days will I get the corrected grad...,Processing of correction on grade cards will t...,Corrections on grade card /PDC
8,Do I need to submit an Exam form for the end t...,"Yes, it is necessary to fill exam form for ev...",Exam Form Issues
9,"I have an internal backlog, do I need to submi...","Yes, it is necessary to fill exam form for in...",Exam Form Issues


In [7]:
text_q  = faq.question[13]
text_a = faq.answer[13]

In [8]:
doc_q = nlp(text_q)
doc_a = nlp(text_a)
for tok in doc_q:
  print(tok.text, '(',tok.dep_,')', '--->', tok.lemma_)

Do ( aux ) ---> do
I ( nsubj ) ---> -PRON-
have ( ROOT ) ---> have
to ( aux ) ---> to
pay ( xcomp ) ---> pay
exam ( compound ) ---> exam
fees ( dobj ) ---> fee
for ( prep ) ---> for
regular ( amod ) ---> regular
and ( cc ) ---> and
backlog ( conj ) ---> backlog
separately ( advmod ) ---> separately
? ( punct ) ---> ?


In [9]:
sen = list(doc.sents)
print(len(sen))
for sens in sen:
  print(sens)

NameError: ignored

In [10]:
from spacy import displacy
displacy.render(doc_a, style='ent', jupyter=True)

In [11]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [13]:
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [15]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [16]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [19]:
from sklearn.model_selection import train_test_split

X = faq['answer'] # the features we want to analyze
ylabels = faq['question'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [20]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7fee57c99690>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7fee68cb8cb0>)),
                ('classifier', LogisticRegression())])

In [21]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.0


ValueError: ignored